<a href="https://colab.research.google.com/github/CoWoGeo/PUS2022_CWolk/blob/main/HW6/HW6_RandomForestsDC_Take_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Washington, DC Building Data

Following the paper "Examining the feasibility of using open data to benchmark building energy usage in cities: A data science and policy perspective" [Roth et al 2019](https://www.researchgate.net/publication/337074109_Examining_the_feasibility_of_using_open_data_to_benchmark_building_energy_usage_in_cities_A_data_science_and_policy_perspective), choose 2 cities in the dataset available through the paper's github repository https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking

This paper is an excellent example of reproducible research. 

I also created an example of using random forest classifier and regressors here https://github.com/fedhere/PUS2022_FBianco/blob/master/classdemo/COVID19LOS_featureExtractionRFexample.ipynb and there are links at the bottom of this notebook to useful functions and examples on the internet of applications of methods you will need to use.

1. Read the introduction to the paper and describe here (briefly) the goals of their analysis (and the analysis that you are about to reproduce). Max 200 words.
2. Choose 2 cities (any 2 cities except Seattle cause I am running some of the Seattle analysis below for guidance)
3. For each of the 2 cities reproduce their analysis by 

  3.1 gathering the original data from their repository (see below for Seattle example)

  3.2 clean the data according to their data preparation scheme, including one-hot-encoding categorical variables, except to impute missing data using KNearestNeighors instead of Gibbs sampling (see below)

  3.3 run a Random Forest (RF) Regressor to predict the total energy consumption.

  3.4 evaluate the RF model performance by printing the R2 score (the default score in SKLearn RF) for training and test set (discuss)

  3.5 plot the features sorted by their importance and identify the most important features. are they consistent between the 2 cities? are they consistent with the paper's result?
4 compare the result for the 2 cities (discuss)

5. Extra credit: 

  5.1 modify the target variable to a binary variable choosing the median of the variable values as the the energy threshold and predict whether the value is higher or lower than the threshold
  5.2 Run a Random Forest Classifier to predict if an observation is in the upper 50% percentile or lower 50% percentile (above or below the median threshold)
  5.3 Measure the performance of the model on training and test data
  5.4 Measure the feature importance and assess if the important feature have changed (discuss)

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pylab as plt

In [3]:
import seaborn as sns

In [4]:
# for imputing data
from sklearn.impute import KNNImputer

In [5]:
# for one-hot encoding
from sklearn.preprocessing import OneHotEncoder
# for train-test split
from sklearn.model_selection import train_test_split
# for random forests
from sklearn.ensemble import RandomForestRegressor

# Reading in DC Data

In [6]:
dc = pd.read_csv("https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/raw/master/DC/2016%20Energy%20and%20Water%20Performance%20Benchmarking%20Results%20as%20of%2009-07-2017.csv")

# inspect the dataframe

In [7]:
print("there are (rows, columns) = ", (dc.shape), "in the dataframe")

there are (rows, columns) =  (2181, 36) in the dataframe


In [8]:
dc.describe()

,pm_pid,ward,year_built,tax_record_floor_area,reported_gross_floor_area,energy_star_score,total_ghg_emissions,total_ghg_emissions_intensity,water_use,electricity_use,natural_gas_use,fuel_use,district_water_use,latitude,longitude
count,1.574000e+03,1844.000000,2153.000000,2.181000e+03,1.521000e+03,1209.000000,1519.000000,1519.000000,1.311000e+03,1.482000e+03,1.087000e+03,6.600000e+01,1.700000e+01,1954.000000,1954.000000
mean,3.477316e+06,3.387202,1933.421273,2.009089e+05,1.876181e+05,61.207610,1397.998947,7.221922,5.582172e+04,3.041003e+06,6.745484e+04,1.133411e+06,-1.261140e+07,38.908558,-77.025736
std,1.151670e+06,1.996951,223.684123,2.447139e+05,2.431177e+05,27.486072,2790.536339,6.886701,6.498499e+05,6.584801e+06,2.975285e+05,2.171472e+06,9.224563e+07,0.025003,0.031201
min,2.214000e+03,1.000000,0.000000,9.171000e+03,5.982000e+03,1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.097600e+03,-3.685268e+08,38.813539,-77.107478
25%,3.300369e+06,2.000000,1942.000000,7.044300e+04,6.960000e+04,42.000000,390.300000,4.800000,2.114100e+03,7.495754e+05,1.115681e+04,2.461230e+04,7.908049e+05,38.898739,-77.044005
50%,3.617946e+06,2.000000,1969.000000,1.213110e+05,1.194700e+05,69.000000,755.400000,6.300000,4.160100e+03,1.656484e+06,2.791756e+04,1.952242e+05,5.512830e+06,38.905369,-77.031106
75%,4.066820e+06,5.000000,1997.000000,2.488440e+05,2.332170e+05,84.000000,1565.150000,7.900000,7.286700e+03,3.491451e+06,5.286096e+04,1.371327e+06,1.457592e+07,38.922848,-77.009371
max,5.939893e+06,8.000000,2017.000000,3.997572e+06,5.634890e+06,100.000000,64992.500000,134.400000,1.929391e+07,1.661020e+08,7.269109e+06,1.275372e+07,3.101497e+07,38.984554,-76.914972


# TASK 1 Clean the data 
*follow closely the second paragraph of Section 4 DATA. Name each step you do accordingly to the description in the paragraf*

Summary of that paragraph:

* Removed irrelevant building-identifying features such as address
*    Removed features w/ missing values for >40% of buildings in dataset because otherwise those will be imputed and bias the results.
*    Eliminated buildings w/ missing site EUI to avoid imputing values for dependent variable
*    Imputed data for remaining missing variables by generating multiple imputations by Gibbs sampling using classification and regression trees (I think this is a later step in the homework.)
*    Authors note that the data is extremely heterogeneous within and between datasets in terms of building counts and building sizes. This is partly due to local mandates, inclusion of government buildings, and smaller buildings opting in in some places. They note that "some models are better able to handle less data than others."

In [9]:
# summarize the number of rows with missing values for each column
for c in dc.columns:
  
  # count number of rows with missing values
  n_miss = dc[c].isnull().sum()
  perc = n_miss / dc.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, dc[c].dtype, 
                                                  n_miss, perc))


pid (object):  Missing: 0 (0.0%)
dc_real_pid (object):  Missing: 0 (0.0%)
pm_pid (float64):  Missing: 607 (27.8%)
property_name (object):  Missing: 660 (30.3%)
pm_parent_pid (object):  Missing: 660 (30.3%)
parent_property_name (object):  Missing: 660 (30.3%)
year_ending (object):  Missing: 0 (0.0%)
report_status (object):  Missing: 0 (0.0%)
address_of_record (object):  Missing: 0 (0.0%)
owner_of_record (object):  Missing: 0 (0.0%)
ward (float64):  Missing: 337 (15.5%)
reported_address (object):  Missing: 660 (30.3%)
city (object):  Missing: 0 (0.0%)
state (object):  Missing: 0 (0.0%)
postal_code (object):  Missing: 607 (27.8%)
year_built (float64):  Missing: 28 (1.3%)
primary_ptype_self (object):  Missing: 660 (30.3%)
primary_ptype_epa (object):  Missing: 660 (30.3%)
tax_record_floor_area (float64):  Missing: 0 (0.0%)
reported_gross_floor_area (float64):  Missing: 660 (30.3%)
energy_star_score (float64):  Missing: 972 (44.6%)
site_eui (object):  Missing: 674 (30.9%)
weather_norm_site_e

this data is so much worse than the final one fml

In [10]:
# dropping columns w/ over 40% missing
for c in dc.columns:
  
  # count number of rows with missing values
  n_miss = dc[c].isnull().sum()
  perc = n_miss / dc.shape[0] * 100
  if perc > 40:
    dc.drop(c, axis=1, inplace=True)

print("there are (rows, columns) = ", (dc.shape), "in the dataframe")

there are (rows, columns) =  (2181, 31) in the dataframe


In [11]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181 entries, 0 to 2180
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   pid                            2181 non-null   object 
 1   dc_real_pid                    2181 non-null   object 
 2   pm_pid                         1574 non-null   float64
 3   property_name                  1521 non-null   object 
 4   pm_parent_pid                  1521 non-null   object 
 5   parent_property_name           1521 non-null   object 
 6   year_ending                    2181 non-null   object 
 7   report_status                  2181 non-null   object 
 8   address_of_record              2181 non-null   object 
 9   owner_of_record                2181 non-null   object 
 10  ward                           1844 non-null   float64
 11  reported_address               1521 non-null   object 
 12  city                           2181 non-null   o

In [12]:
# Dropping columns w/ building identifying data
dc = dc.drop(["pid", "dc_real_pid", "pm_pid", "property_name",
              "pm_parent_pid", "parent_property_name", "address_of_record",
              "reported_address"], axis=1)

In [13]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181 entries, 0 to 2180
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year_ending                    2181 non-null   object 
 1   report_status                  2181 non-null   object 
 2   owner_of_record                2181 non-null   object 
 3   ward                           1844 non-null   float64
 4   city                           2181 non-null   object 
 5   state                          2181 non-null   object 
 6   postal_code                    1574 non-null   object 
 7   year_built                     2153 non-null   float64
 8   primary_ptype_self             1521 non-null   object 
 9   primary_ptype_epa              1521 non-null   object 
 10  tax_record_floor_area          2181 non-null   float64
 11  reported_gross_floor_area      1521 non-null   float64
 12  site_eui                       1507 non-null   o

In [14]:
# dropping city, state bc we know those are DC
dc = dc.drop(["city", "state"], axis=1)

In [15]:
dc["year_ending"].nunique()

1

In [16]:
# dropping year ending because it's all the same
dc = dc.drop(["year_ending"], axis=1)

In [17]:
# what is report status?
dc["report_status"]

0                   In Compliance
1                   In Compliance
2                   In Compliance
3                   In Compliance
4                   In Compliance
                  ...            
2176    Data under review by DOEE
2177    Data under review by DOEE
2178    Data under review by DOEE
2179    Data under review by DOEE
2180    Data under review by DOEE
Name: report_status, Length: 2181, dtype: object

In [132]:
# dropping site EUI Nas because that's what the authors did
dc.dropna(subset=['site_eui'], inplace=True)

# TASK 2: transform input feature as needed
for example replace features with log features. Guide yourself with the text, the final dataset names, and also the code here https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd

In the describe function below, I'm mostly looking at min, max, and standard deviation to get a sense of how big the ranges are and if there are columns that should have a log function or have very little variance.

In [133]:
dc.describe() #what should you look at in the result below?

,ward,year_built,tax_record_floor_area,reported_gross_floor_area,site_eui,total_ghg_emissions,total_ghg_emissions_intensity,water_use,electricity_use,latitude,longitude,BuildingEnergyUse
count,1171.000000,1503.000000,1.506000e+03,1.506000e+03,1506.000000,1504.000000,1504.000000,1.296000e+03,1.482000e+03,1444.000000,1444.000000,1506.000000
mean,3.100769,1940.258816,2.059340e+05,1.886712e+05,79.634595,1411.941755,7.293949,5.642857e+04,3.041003e+06,38.908326,-77.026829,-inf
std,1.821549,168.258449,2.483237e+05,2.440733e+05,144.859256,2800.912178,6.882896,6.535778e+05,6.584801e+06,0.024269,0.029901,NaN
min,1.000000,1000.000000,9.171000e+03,5.982000e+03,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,38.813539,-77.100527,-inf
25%,2.000000,1945.000000,7.139625e+04,7.000350e+04,51.425000,396.875000,4.800000,2.168150e+03,7.495754e+05,38.898738,-77.044200,6.653792
50%,2.000000,1970.000000,1.287695e+05,1.199814e+05,66.300000,775.200000,6.300000,4.182900e+03,1.656484e+06,38.904623,-77.031576,6.920291
75%,5.000000,1994.000000,2.581805e+05,2.349805e+05,84.000000,1584.650000,8.000000,7.320475e+03,3.491451e+06,38.921214,-77.011514,7.202882
max,8.000000,2017.000000,3.997572e+06,5.634890e+06,5059.200000,64992.500000,134.400000,1.929391e+07,1.661020e+08,38.983573,-76.919927,8.893900


In [134]:
dc.info() #why is this helpful to print?

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   report_status                  1506 non-null   object 
 1   owner_of_record                1506 non-null   object 
 2   ward                           1171 non-null   float64
 3   postal_code                    1506 non-null   object 
 4   year_built                     1503 non-null   float64
 5   primary_ptype_self             1506 non-null   object 
 6   primary_ptype_epa              1506 non-null   object 
 7   tax_record_floor_area          1506 non-null   float64
 8   reported_gross_floor_area      1506 non-null   float64
 9   site_eui                       1506 non-null   float64
 10  weather_norm_site_eui          1444 non-null   object 
 11  source_eui                     1506 non-null   object 
 12  weather_norm_source_eui        1444 non-null   o

In [214]:
dc["reported_gross_floor_area"].describe()

count    1.506000e+03
mean     1.886712e+05
std      2.440733e+05
min      5.982000e+03
25%      7.000350e+04
50%      1.199814e+05
75%      2.349805e+05
max      5.634890e+06
Name: reported_gross_floor_area, dtype: float64

In [213]:
dc["site_eui"].describe()

count    1506.000000
mean       79.634595
std       144.859256
min         0.000000
25%        51.425000
50%        66.300000
75%        84.000000
max      5059.200000
Name: site_eui, dtype: float64

In [136]:
dc["source_eui"]

0       175.9
1       229.8
2       215.6
3       147.7
4       206.3
        ...  
2164    175.2
2167     13.8
2171        0
2179      0.1
2180     16.4
Name: source_eui, Length: 1506, dtype: object

Idk what source EUI is, but their paper seems to use site EUI vs source EUI and based on Chicago, I think I should multiply by gross area, not tax reported floor area. (I assume this is like how my house is not taxed on the finished basement as rooms?)

In [219]:
#realized I need to make site eui a float.
dc["site_eui"] = pd.to_numeric(dc["site_eui"], errors='coerce')

In [220]:
#adding column for building energy use
dc["BuildingEnergyUse"] = dc["reported_gross_floor_area"]*dc["site_eui"]

In [221]:
dc["BuildingEnergyUse"].describe()

count    1.506000e+03
mean     1.511370e+07
std      3.874259e+07
min      0.000000e+00
25%      4.506007e+06
50%      8.323210e+06
75%      1.595444e+07
max      7.832497e+08
Name: BuildingEnergyUse, dtype: float64

In [218]:
#making building energy use log
dc["BuildingEnergyUse"] = np.log10(dc["BuildingEnergyUse"])

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [140]:
X_numeric_data["tax_record_floor_area"] = np.log10(X_numeric_data["tax_record_floor_area"])

In [141]:
dc["BuildingEnergyUse"].describe()

count    1506.000000
mean            -inf
std              NaN
min             -inf
25%         6.653792
50%         6.920291
75%         7.202882
max         8.893900
Name: BuildingEnergyUse, dtype: float64

In [142]:
np.isinf(dc["BuildingEnergyUse"])

0       False
1       False
2       False
3       False
4       False
        ...  
2164    False
2167    False
2171     True
2179    False
2180    False
Name: BuildingEnergyUse, Length: 1506, dtype: bool

## Throw away data where the **target** variable is missing

I will remove each row where I do not have the target variable. To do that I use `df.dropna()` and the subset argument of `df.dropna()` set to the name of the variable. The `how` should be set to "any" cause you want to drop the row where any values in the subset is NaN (this is a subset of one column, so it is obvious, but imagine if you were passing more than one column to the call `dropna()`

In [145]:
# one is the loneliest number
dc["BuildingEnergyUse"].isna().sum()

0

In [146]:
# oops had forgotten to drop na earlier
dc.dropna(subset=["BuildingEnergyUse"], how="any", inplace=True)

In [147]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   report_status                  1506 non-null   object 
 1   owner_of_record                1506 non-null   object 
 2   ward                           1171 non-null   float64
 3   postal_code                    1506 non-null   object 
 4   year_built                     1503 non-null   float64
 5   primary_ptype_self             1506 non-null   object 
 6   primary_ptype_epa              1506 non-null   object 
 7   tax_record_floor_area          1506 non-null   float64
 8   reported_gross_floor_area      1506 non-null   float64
 9   site_eui                       1506 non-null   float64
 10  weather_norm_site_eui          1444 non-null   object 
 11  source_eui                     1506 non-null   object 
 12  weather_norm_source_eui        1444 non-null   o

In [148]:
original_len = dc.shape[0]
dc.dropna(subset=["BuildingEnergyUse"], how="any", inplace=True)

print("I lost {:.2f}% of the data".format((1 - (dc.shape[0] / original_len)) * 100))

I lost 0.00% of the data


In [149]:
# isolate the target variable first (endogenous)
y = dc["BuildingEnergyUse"].values

In [150]:
# and the input variables (exogenous)
X = dc.drop("BuildingEnergyUse", axis=1)

In [151]:
dc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   report_status                  1506 non-null   object 
 1   owner_of_record                1506 non-null   object 
 2   ward                           1171 non-null   float64
 3   postal_code                    1506 non-null   object 
 4   year_built                     1503 non-null   float64
 5   primary_ptype_self             1506 non-null   object 
 6   primary_ptype_epa              1506 non-null   object 
 7   tax_record_floor_area          1506 non-null   float64
 8   reported_gross_floor_area      1506 non-null   float64
 9   site_eui                       1506 non-null   float64
 10  weather_norm_site_eui          1444 non-null   object 
 11  source_eui                     1506 non-null   object 
 12  weather_norm_source_eui        1444 non-null   o

### separate categorical and numerical values

When I ran the separator, ward showed up in numeric because it is a number, so I converted it to object. All the EUIs were objects and ended up in categorical, so I converted them to numeric.

In [152]:
X["weather_norm_site_eui"] = pd.to_numeric(X["weather_norm_site_eui"], errors='coerce')

In [153]:
X["source_eui"] = pd.to_numeric(X["source_eui"], errors = "coerce")
X["weather_norm_source_eui"] = pd.to_numeric(X["weather_norm_source_eui"], errors = "coerce")

In [154]:
X[["ward"]] = X[["ward"]].astype(object)

In [155]:
# I am giving you the solution but please take note of how one does this!
X_numeric_data = X.select_dtypes(include=[np.number])
X_categorical_data = X.select_dtypes(exclude=[np.number])

In [156]:
X_numeric_data.shape

(1506, 13)

In [157]:
X_numeric_data.head() ## is there anything suspicious? anything thas is in fact **not** a numerical variable? that will depend on how you did in TASK 1

,year_built,tax_record_floor_area,reported_gross_floor_area,site_eui,weather_norm_site_eui,source_eui,weather_norm_source_eui,total_ghg_emissions,total_ghg_emissions_intensity,water_use,electricity_use,latitude,longitude
0,1959.0,63227.0,65000.0,101.3,102.7,175.9,174.9,482.9,7.4,6963.5,633043.8,38.963597,-77.033848
1,1997.0,251557.0,257943.0,73.2,73.2,229.8,229.8,2164.4,8.4,8490.0,5531486.0,38.897849,-77.029872
2,1964.0,330550.0,223218.0,87.8,89.0,215.6,215.0,1852.2,8.3,8749.1,3863003.0,38.904056,-77.037973
3,1969.0,256839.0,213067.0,47.0,46.4,147.7,145.8,1249.2,5.9,5393.8,2936445.0,38.902189,-77.041238
4,1975.0,370000.0,381518.0,65.7,65.7,206.3,206.3,2874.1,7.5,10422.6,7345290.0,38.904376,-77.046882


In [158]:
X_categorical_data.head(3)

,report_status,owner_of_record,ward,postal_code,primary_ptype_self,primary_ptype_epa,metered_areas_energy
0,In Compliance,AUBINOE ROCKVIEW LIMITED PARTNERSHP; VICTORY I...,4.0,20011,Multifamily Housing,Multifamily Housing,Whole Building
1,In Compliance,UNION INVESTMENT REAL ESTATE GMBH,2.0,20005,Office,Office,Whole Building
2,In Compliance,CESC 1101 17TH STREET LIMITED PARTNERSHIP,2.0,20036,Office,Office,Whole Building


In [159]:
X_categorical_data.shape

(1506, 7)

## TASK 2.1: make a scatter plot to assess covariance of numerical variables

### IMPORTANT!! 

there are going to be some **very** collinear variable - zoom in and chek them out - what are they? You will need to remove variables that are _identical_, because you obviously do not want to imput the data twice in the model - it will not improve the model and it will impact the feature importance analysis.

What else looks like it should be removed? 

**Reason about it and explain your choices in the caption of the figure**



In [160]:
# THIS TAKES A LOOOONG TIME! ~10 MINUTES ON COLAB FOR ME
# I deleted output by putting it in #, is that what you meant?
#sns.pairplot(X_numeric_data)

I didn't look at the plots as much this time around because a lot of the columns were the same as the ones I'd decided to delete for Chicago/first pass at DC. I did keep lat/long just to make it spicy this time.

It's interesting: this version of the DC data must have had the natural gas values added or fixed because I dropped it for having 50% missing above, but the other version of this data set still had it here. This set also doesn't have energy star score or total site energy kbtu.

In [161]:
# checking if maybe I should have used this data? but it wouldn't import so too bad
#moredcdata = pd.read_csv("https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/DC/dc_energy_geocoded.csv")

Dr. Bianco and I talked about what to drop! I deleted ALL the EUI ones (including "weather-normalized") electricity use, gas use, and GHG measures because they are basically all redundant or intra-dependent measures of energy use. 

In [162]:
# which rows should you drop?
X_numeric_data.drop(["electricity_use", "site_eui", "weather_norm_site_eui", 
                     "source_eui", "weather_norm_source_eui", 
                     "total_ghg_emissions_intensity", "total_ghg_emissions"], 
                    axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [163]:
X_numeric_data.describe()

,year_built,tax_record_floor_area,reported_gross_floor_area,water_use,latitude,longitude
count,1503.000000,1.506000e+03,1.506000e+03,1.296000e+03,1444.000000,1444.000000
mean,1940.258816,2.059340e+05,1.886712e+05,5.642857e+04,38.908326,-77.026829
std,168.258449,2.483237e+05,2.440733e+05,6.535778e+05,0.024269,0.029901
min,1000.000000,9.171000e+03,5.982000e+03,0.000000e+00,38.813539,-77.100527
25%,1945.000000,7.139625e+04,7.000350e+04,2.168150e+03,38.898738,-77.044200
50%,1970.000000,1.287695e+05,1.199814e+05,4.182900e+03,38.904623,-77.031576
75%,1994.000000,2.581805e+05,2.349805e+05,7.320475e+03,38.921214,-77.011514
max,2017.000000,3.997572e+06,5.634890e+06,1.929391e+07,38.983573,-76.919927


In [164]:
# this does not have to be identical to my output as you may have dropped different variables
print("there are (rows, columns) = ", X_numeric_data.shape, "in the numerical variables")

there are (rows, columns) =  (1506, 6) in the numerical variables


As they do in the original research https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd we should transform some variables into their log values 

their original code (which is in R not in python!) is
```
x_seattle$log_total_area=log(x_seattle$gross_floor_area_total_SF)
x_seattle$log_parking_area=log(x_seattle$gross_floor_area_parking_SF)
x_seattle$log_bldg_area=log(x_seattle$gross_floor_area_building_SF)
x_seattle$log_area_1=log(x_seattle$gross_floor_area_property_type_first_SF)

```
the names are a bit different


In [165]:
# note: recheck missing values 
# this is important: after I apply the log function to some variables cause log(0) = -infinity
print("there are {} missing or infinity values in the numerical data".format(X_numeric_data.isnull().sum().sum()))

there are 337 missing or infinity values in the numerical data


In [166]:
# translated to python
X_numeric_data["tax_record_floor_area"] = np.log10(X_numeric_data["tax_record_floor_area"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [167]:
X_numeric_data["reported_gross_floor_area"] = np.log10(X_numeric_data["reported_gross_floor_area"])
X_numeric_data["water_use"] = np.log10(X_numeric_data["water_use"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [168]:
print("number of infinite values:", (np.isinf(X_numeric_data)).sum().sum())

number of infinite values: 4


check what happened to missing values: they may have grown! because  log(0) = -infinity - fix infinities replacing them with NaN 

(in in https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd they had replaced them with 0 but I object to that)

In [169]:
# this has to return 0 now
X_numeric_data.replace(-np.inf, np.nan, inplace=True)
print("number of infinite values:", (np.isinf(X_numeric_data)).sum().sum())

number of infinite values: 0


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [170]:
print("the numerical data contains (rows, columns) = ", X_numeric_data.shape)
print("there are {} missing values in the numerical data".format(X_numeric_data.isnull().sum().sum()))

the numerical data contains (rows, columns) =  (1506, 6)
there are 341 missing values in the numerical data


In [171]:
#the missing data is in water use
X_numeric_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year_built                 1503 non-null   float64
 1   tax_record_floor_area      1506 non-null   float64
 2   reported_gross_floor_area  1506 non-null   float64
 3   water_use                  1292 non-null   float64
 4   latitude                   1444 non-null   float64
 5   longitude                  1444 non-null   float64
dtypes: float64(6)
memory usage: 82.4 KB


So I am mostly missing data for water use, but also quite a few for lat/long.


you can use `KNNImputer` on the numerical variables, but KNNImputer would fail on categorical variables so you are left with NaNs on categorical variables, which you then need to drop before you run the Random Forest


You can impute separately the numerical and categorical variables, the numerical ones with KNNImputer and the categorical ones with SimpleImputer

## TASK 2.2 use Nearest Neighbor Imputer to impute missing values in numerical features

here we are deviating from the paper. NN imputation is simpler and more common, I think it may be more useful to you in general

In [172]:
imputer = KNNImputer()
Xn = imputer.fit_transform(X_numeric_data)
Xn

array([[1959.        ,    4.80090258,    4.81291336,    3.84282758,
          38.96359693,  -77.033848  ],
       [1997.        ,    5.40063641,    5.41152375,    3.92890769,
          38.89784861,  -77.02987163],
       [1964.        ,    5.51923716,    5.34872921,    3.94196338,
          38.90405585,  -77.03797318],
       ...,
       [1960.        ,    6.03952317,    5.00838723,    4.29946589,
          38.924529  ,  -77.020981  ],
       [1952.        ,    5.21046299,    4.99500303,    2.96241669,
          38.921564  ,  -77.021498  ],
       [1994.        ,    5.2799269 ,    5.35218252,    4.11694628,
          38.920882  ,  -77.017807  ]])

In [173]:
numeric_whatswrong = pd.DataFrame(Xn)

In [174]:
numeric_whatswrong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1506 entries, 0 to 1505
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1506 non-null   float64
 1   1       1506 non-null   float64
 2   2       1506 non-null   float64
 3   3       1506 non-null   float64
 4   4       1506 non-null   float64
 5   5       1506 non-null   float64
dtypes: float64(6)
memory usage: 70.7 KB


In [175]:
print("number of infinite values:", (np.isinf(numeric_whatswrong)).sum().sum())

number of infinite values: 0


In [176]:
numeric_whatswrong.describe()

,0,1,2,3,4,5
count,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000,1506.000000
mean,1940.352988,5.135574,5.105672,3.599606,38.908373,-77.026888
std,168.103973,0.389268,0.377645,0.616912,0.023891,0.029374
min,1000.000000,3.962417,3.776846,-0.221849,38.813539,-77.100527
25%,1945.250000,4.853675,4.845120,3.344072,38.898746,-77.043871
50%,1970.500000,5.109813,5.079114,3.624375,38.904656,-77.031404
75%,1994.000000,5.411923,5.371032,3.885368,38.920870,-77.012701
max,2017.000000,6.601796,6.750885,7.285420,38.983573,-76.919927


## TASK 2.3 Impute missing categorial variables

Ugh sooo many are missing now

In [177]:
print("there are {} missing  values in the categorical data".format(X_categorical_data.isna().sum().sum()))

there are 444 missing  values in the categorical data


In [178]:
X_categorical_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   report_status         1506 non-null   object
 1   owner_of_record       1506 non-null   object
 2   ward                  1171 non-null   object
 3   postal_code           1506 non-null   object
 4   primary_ptype_self    1506 non-null   object
 5   primary_ptype_epa     1506 non-null   object
 6   metered_areas_energy  1397 non-null   object
dtypes: object(7)
memory usage: 94.1+ KB


I'm going to fill the missing metered_areas_energy with "None" because there are just enough missing that I'd be worried about imputing them without making the data worse.

In [179]:
# filling metered areas energy with None
X_categorical_data["metered_areas_energy"].fillna("None", inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [180]:
X_categorical_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 2180
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   report_status         1506 non-null   object
 1   owner_of_record       1506 non-null   object
 2   ward                  1171 non-null   object
 3   postal_code           1506 non-null   object
 4   primary_ptype_self    1506 non-null   object
 5   primary_ptype_epa     1506 non-null   object
 6   metered_areas_energy  1506 non-null   object
dtypes: object(7)
memory usage: 94.1+ KB


In [181]:
Xc_nostringz = X_categorical_data[["ward", "postal_code"]]

In [182]:
Xc_nostringz["ward"] = pd.to_numeric(Xc_nostringz["ward"], errors = "coerce")
Xc_nostringz["postal_code"] = pd.to_numeric(Xc_nostringz["postal_code"], errors = "coerce")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [183]:
imputer = KNNImputer(add_indicator=True, n_neighbors=3)
Xc_nostringz = imputer.fit_transform(Xc_nostringz)
Xc_nostringz

array([[4.0000e+00, 2.0011e+04, 0.0000e+00, 0.0000e+00],
       [2.0000e+00, 2.0005e+04, 0.0000e+00, 0.0000e+00],
       [2.0000e+00, 2.0036e+04, 0.0000e+00, 0.0000e+00],
       ...,
       [1.0000e+00, 2.0059e+04, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.0059e+04, 0.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.0059e+04, 0.0000e+00, 0.0000e+00]])

In [184]:
Xc_nostringz = pd.DataFrame(Xc_nostringz)
Xc_nostringz.rename({0: "ward", 1: 'postal_code'}, axis=1, inplace=True)
Xc_nostringz.drop(columns=[2], inplace=True)

In [185]:
Xc_nostringz.drop(columns=[3], inplace=True)

In [186]:
Xc_nostringz

,ward,postal_code
0,4.0,20011.0
1,2.0,20005.0
2,2.0,20036.0
3,2.0,22202.0
4,2.0,20037.0
...,...,...
1501,5.0,20017.0
1502,1.0,20059.0
1503,1.0,20059.0
1504,1.0,20059.0


In [187]:
Xc_nostringz = Xc_nostringz.astype("int")

In [188]:
Xc_nostringz["ward"]

0       4
1       2
2       2
3       2
4       2
       ..
1501    5
1502    1
1503    1
1504    1
1505    1
Name: ward, Length: 1506, dtype: int64

Looking at this again, I see that there are many more postal codes than wards. But this might be good, since I have fewer possibilities in the missing values than the data I'm using to fill them in.

In [189]:
Xc_nostringz.nunique()

ward            8
postal_code    44
dtype: int64

In [190]:
Xc_nostringz.head()

,ward,postal_code
0,4,20011
1,2,20005
2,2,20036
3,2,22202
4,2,20037


In [191]:
Xc_nostringz[["ward"]] = Xc_nostringz[["ward"]].astype(object)
Xc_nostringz[["postal_code"]] = Xc_nostringz[["postal_code"]].astype(object)

In [192]:
X_cat = X_categorical_data.drop(["ward"], axis=1)

In [193]:
X_cat.reset_index()

,index,report_status,owner_of_record,postal_code,primary_ptype_self,primary_ptype_epa,metered_areas_energy
0,0,In Compliance,AUBINOE ROCKVIEW LIMITED PARTNERSHP; VICTORY I...,20011,Multifamily Housing,Multifamily Housing,Whole Building
1,1,In Compliance,UNION INVESTMENT REAL ESTATE GMBH,20005,Office,Office,Whole Building
2,2,In Compliance,CESC 1101 17TH STREET LIMITED PARTNERSHIP,20036,Office,Office,Whole Building
3,3,In Compliance,1776 K STREET ASSOCIATES & RIDDELL PARTNERS,22202,Office,Office,Whole Building
4,4,In Compliance,CESC 2101 L STREET LLC,20037,Office,Office,Whole Building
...,...,...,...,...,...,...,...
1501,2164,In Compliance,Monroe Street Market - Cornerstone,20017,Multifamily Housing,Multifamily Housing,Whole Building
1502,2167,Data under review by DOEE,HOWARD UNIVERSITY (LAND ONLY),20059,College/University,Other,Whole Building
1503,2171,Data under review by DOEE,HOWARD UNIVERSITY (LAND ONLY),20059,College/University,College/University,Whole Building
1504,2179,Data under review by DOEE,HOWARD UNIVERSITY (LAND ONLY),20059,College/University,College/University,Whole Building


In [194]:
X_cat.head()

,report_status,owner_of_record,postal_code,primary_ptype_self,primary_ptype_epa,metered_areas_energy
0,In Compliance,AUBINOE ROCKVIEW LIMITED PARTNERSHP; VICTORY I...,20011,Multifamily Housing,Multifamily Housing,Whole Building
1,In Compliance,UNION INVESTMENT REAL ESTATE GMBH,20005,Office,Office,Whole Building
2,In Compliance,CESC 1101 17TH STREET LIMITED PARTNERSHIP,20036,Office,Office,Whole Building
3,In Compliance,1776 K STREET ASSOCIATES & RIDDELL PARTNERS,22202,Office,Office,Whole Building
4,In Compliance,CESC 2101 L STREET LLC,20037,Office,Office,Whole Building


In [195]:
Xc = X_cat.join(Xc_nostringz, rsuffix="right")

In [196]:
Xc2 = X_cat.merge(Xc_nostringz, right_index=True, left_index=True)

#Where Coryn was Stuck

Why did it only impute ~60 of my missing wards? Did the other ones not have enough info? Where did all those zip codes go?

# How Fed Fixed It
Tho we still don't know why it was either giving me 1507 rows with Nas for some of the wards and one column of zipcodes (though fewer Na wards than the original!) or giving me full rows but fewer than 1507 for everything.

In [197]:
Xc_nostringz.index = list(range(Xc_nostringz.shape[0]))
X_cat.index = list(range(Xc_nostringz.shape[0]))

In [198]:
X_cat = X_cat.join(Xc_nostringz, rsuffix="right")

In [199]:
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506 entries, 0 to 1505
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   report_status         1506 non-null   object
 1   owner_of_record       1506 non-null   object
 2   postal_code           1506 non-null   object
 3   primary_ptype_self    1506 non-null   object
 4   primary_ptype_epa     1506 non-null   object
 5   metered_areas_energy  1506 non-null   object
 6   ward                  1506 non-null   object
 7   postal_coderight      1506 non-null   object
dtypes: object(8)
memory usage: 138.2+ KB


# TASK 3 One Hot Encode the Categorical Variables

Hint: Once you have done the fit and the transformation, which gives you the transformed data according to the new encoding, what you get is normally a sparse matrix. In principle there is an argument `sparse` to `OneHotEncoder` which should prevent this and give you a dense matrix, which is much easier to handle (consider asking me or looking up what is a sparse matrix!). That attribute did not work for me so once I applied the new encoding I converted the result to a dense matrix as 
`Xc = Xc.todense()`

Finally, you will need to use a specific method to get the new names of the variables. Its a method of your model (which I called ohe below) and you can find it in this page here https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [200]:
X_cat.describe()

,report_status,owner_of_record,postal_code,primary_ptype_self,primary_ptype_epa,metered_areas_energy,ward,postal_coderight
count,1506,1506,1506,1506,1506,1506,1506,1506
unique,2,1088,52,46,46,4,8,44
top,In Compliance,GOVERNMENT OF THE DISTRICT OF COLUMBIA DEPARTM...,20005,Office,Office,Whole Building,2,20005
freq,1436,198,142,504,496,1372,629,142


Ooops where I went wrong AGAIN was one hot encoding the original data, not the fixed data!

In [201]:
# One Hot Encoding
ohe = OneHotEncoder(sparse=False).fit(X_cat)
Xc = ohe.transform(X_cat)
Xc

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [202]:
#get the names of the features after the encoding is done
feature_name_cat = ohe.get_feature_names_out()
feature_name_cat

array(['report_status_Data under review by DOEE',
       'report_status_In Compliance',
       'owner_of_record_"11TH & G ST INVESTORS', ...,
       'postal_coderight_200171566', 'postal_coderight_200242459',
       'postal_coderight_200362103'], dtype=object)

In [203]:
Xc.shape

(1506, 1290)

# Task 4 recombine your numerical and categorical imput features and split your data into training and testing

In [204]:
Xn.shape

(1506, 6)

In [205]:
Xc.shape

(1506, 1290)

In [206]:
X = np.hstack([Xn, Xc])
X.shape

(1506, 1296)

In [207]:
print("The dataset has {} features (!!!)".format(X.shape[1]))

The dataset has 1296 features (!!!)


In [208]:
#testing Xn for error sources
Xn_train, Xn_test, y_train, y_test = train_test_split(Xn, y, train_size=0.70, random_state=72)
Xn_train.shape, Xn_test.shape, y_train.shape, y_test.shape

((1054, 6), (452, 6), (1054,), (452,))

In [209]:
#testing Xc for error sources
Xc_train, Xc_test, y_train, y_test = train_test_split(Xc, y, train_size=0.70, random_state=72)
Xc_train.shape, Xc_test.shape, y_train.shape, y_test.shape

((1054, 1290), (452, 1290), (1054,), (452,))

In [210]:
#I tried a few different max depths here.
rf = RandomForestRegressor(max_depth=4)
rf.fit(Xc_train, y_train)

ValueError: ignored

In [211]:
# make a train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=72)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1054, 1296), (452, 1296), (1054,), (452,))

Tbh I didn't mess with the train and random state settings this time because I need to move on with my life.

# TASK 5 run a random forest REGRESSION model

report the score on the training and test data and identify the most important features

In [212]:
#I tried a few different max depths here.
rf = RandomForestRegressor(max_depth=4)
rf.fit(X_train, y_train)
print("accuracy on training data {:.2}".format(rf.score(X_train, y_train)))

ValueError: ignored

# Model Results and Analysis

In [ ]:
print ("the model test accuracy is {:.2}".format(rf.score(X_test, y_test)))

Oooh this one has a much worse test vs training performance than my Chicago one!

In [ ]:
#this is how you see the importance of the features
rf.feature_importances_[:10]

In [ ]:
# One thing I tried
#feature_names = list(X_train.columns.values)

In [ ]:
# making updated array of feature/column names (numeric then categorical)
columns = np.concatenate([X_numeric_data.columns, feature_name_cat])
#columns

In [ ]:
#creating an index of rankings of feature importances
sorted_idx = np.argsort(rf.feature_importances_)

In [ ]:
sorted_column_names = columns[sorted_idx]

In [ ]:
len(columns)

In [ ]:
# plot the top ~50 features
y_ticks = np.arange(0, len(columns))
fig, ax = plt.subplots(figsize=(10,20))
ax.barh(y_ticks, rf.feature_importances_[sorted_idx])
ax.set_yticklabels(sorted_column_names)
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
ax.set_ylim(112, len(columns))
plt.show()

## Conclusion

Like in Chicago and the authors' findings, building area is important! It looks like my findings are pretty similar to the authors'. I'm not actually sure what tax recorded area is...is it a variation on the same data or taxable area? Unlike Chicago, this one has water use, and I think the metered water variable is the one the authors said can serve as a proxy for conditioned/frequently used by people areas. Unlike Chicago, churches and grocery stores don't show up as important here--were they outweighed by other variables? Museums do--DC does have a lot of big museums, so maybe they appear enough in the data to stand out.

I looked up zip codes 20059 and 20024. 20059 is pretty small but has some of Howard University and a stadium. 20024 has a lot of greenspace, but it's next to Capitol Hill and the Mall and incorporates the Smithsonian. Based on the museums variable, I wonder if the large amount of museums make it stand out? Are museums really that much bigger energy hogs than huge office buildings?

# useful links and methods and functions I used: 

```df.dropna()``` to remove observation with missing variables

```df.drop(columns, axis=1)``` to remove features (columns)


```np.log10()``` and ```np.isinf()``` to convert to log space and find infinite values  

`np.hstack([X1, X2])` to concatenate two arrays


one hot encoder https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html and https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/


KNN imputer https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html , https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/, and 
https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/  